In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import json

In [2]:
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter
import pytorch_lightning as pl

In [3]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

In [4]:
def feature_engineer(data):
    """
    Arguments:
    data: pandas.DataFrame that must have specific columns.

    """
    # Bid-Ask spread: (Ask - Bid) / Ask
    data["best_bid"] = (data["best_offer"] - data["best_bid"]) / (data["best_offer"])
    data = data.rename(columns={"best_bid": "ba_spread_option"}).drop(["best_offer"], axis=1)

    # Gamma: multiply by spotprice and divide by 100
    data["gamma"] = data["gamma"] * data["spotprice"] / 100 #following Bali et al. (2021)

    # Theta: scale by spotprice
    data["theta"] = data["theta"] / data["spotprice"] #following Bali et al. (2021)

    # Vega: scale by spotprice
    data["vega"] = data["vega"] / data["spotprice"] #following Bali et al. (2021)

    # Time to Maturity: cale by number of days in year: 365
    data["days_to_exp"] = data["days_to_exp"] / 365

    # Moneyness: Strike / Spot (K / S)
    data["strike_price"] = data["strike_price"] / data["spotprice"] # K / S
    data = data.rename(columns={"strike_price": "moneyness"})

    # Forward Price ratio: Forward / Spot
    data["forwardprice"] = data["forwardprice"] / data["spotprice"]

    # Drop redundant/ unimportant columns
    data = data.drop(["cfadj", "days_no_trading", "spotprice", "adj_spot"], axis=1)

    return data

In [5]:
# binary y label generator
def binary_categorize(y):
    """
    Input: continuous target variable 

    Output: 1 for positive returns, 
            0 for negative returns
    """
    if y > 0:
        return 1
    else:
        return 0


# multiclass y label generator
def multi_categorize(y):
    """
    Input: continuous target variable
    CAREFUL: classes have to be between [0, C) for F.crossentropyloss.
    
    Output: multi class
    """
    if y > 0.05:
        return 2
    elif y < -0.05:
        return 0
    else:
        return 1

In [6]:
class CVSplitter:
    """ Generator for data splits
    Args:
    dates: pandas.Series of datetime,
    init_train_length: int,
    val_length: int
    """
    def __init__(self, dates, init_train_length=10, val_length=2, test_length=1):
        # find indeces where years change (will ignore last year end in dates)
        self.val_length = val_length
        self.test_length = test_length
        self.eoy_idx =  np.where((dates.dt.year.diff() == 1))[0]
        self.eoy_idx = np.append(self.eoy_idx, len(dates)) #append end of year of last year in dates

        assert init_train_length + val_length + test_length <= len(self.eoy_idx) + 1, \
        "defined train and val are larger than number of years in dataset"
        assert init_train_length > 0, "init_train_length must be strictly greater than 0"

        # align the 4th idx to be the end of the 5th year...
        self.train_start_idx = init_train_length - 1

        self.train_eoy = self.eoy_idx[self.train_start_idx:-(val_length+test_length)]
        self.val_eoy = self.eoy_idx[self.train_start_idx + val_length:-test_length]
        # For generate_idx():
        self.test_eoy = self.eoy_idx[self.train_start_idx + val_length + test_length:]

    def generate(self):
        for i in range(len(self.eoy_idx) - (self.train_start_idx + self.val_length)):
            yield (list(range(self.train_eoy[i])),
                   list(range(self.train_eoy[i], self.val_eoy[i])))

    def generate_idx(self):
        for i in range(len(self.eoy_idx) - (self.train_start_idx + self.val_length 
                        + self.test_length)):
            yield ({"train": self.train_eoy[i], 
                    "val": self.val_eoy[i], 
                    "test": self.test_eoy[i]}
                )

In [7]:
# read data from disk
path = Path(r"C:\Users\Mathiass\OneDrive - Universität Zürich UZH\Documents\mt_literature\data")

class Dataset():
    def __init__(self, path=path, year_idx=0, dataset="small", init_train_length=23, val_length=2, label_fn="binary"):
        if dataset == "small":
            self.data = pd.read_parquet(path/"final_df_filledmean_small.parquet")
        elif dataset == "big":
            self.data = pd.read_parquet(path/"final_df_filledmean.parquet")
        else:
            raise ValueError("Specify dataset as either 'small' or 'big'")

        # get splits
        splitter = CVSplitter(self.data["date"], init_train_length=init_train_length, 
                                val_length=val_length, test_length=1)
        eoy_indeces = list(splitter.generate_idx())
        self.eoy_train = eoy_indeces[year_idx]["train"]
        self.eoy_val = eoy_indeces[year_idx]["val"]
        self.eoy_test = eoy_indeces[year_idx]["test"]
        
        # Truncate data
        self.data = self.data.iloc[:self.eoy_test]
        assert len(self.data) == self.eoy_test, "length of data is not equal to eoy_test"
            
        # feature engineer data
        self.data = feature_engineer(self.data)
        
        # create y
        self.y = self.data["option_ret"]
        # make classification problem
        if label_fn == "binary":
            self.y = self.y.apply(binary_categorize)
        elif label_fn == "multi":
            self.y = self.y.apply(multi_categorize)
        else:
            raise ValueError("Specify label_fn as either 'binary' or 'multi'")
        # create X
        self.X = self.data.drop(["option_ret"], axis=1)
        
        # save dates and drop
        self.dates = self.X["date"]
        self.X = self.X.drop(["date"], axis=1)
        
#         # to torch Tensor
#         self.X = torch.from_numpy(self.X.values).float() #-> will be standardized in setup, so do it there.
#         self.y = torch.from_numpy(self.y.values)

        # to numpy
        self.X = self.X.values #-> will be standardized in setup, so do it there.
        self.y = self.y.values
    
        ############################### setup #########################################################
        # train
        # self.X_train = self.X[self.dates < self.hparams.start_val]
        self.X_train = self.X[:self.eoy_train]
        self.y_train = self.y[:len(self.X_train)]
        
        #val
        # mask = (self.dates >= self.hparams.start_val) & (self.dates < self.hparams.start_test)
        # self.X_val = self.X[mask]
        self.X_val = self.X[self.eoy_train:self.eoy_val]
        self.y_val = self.y[len(self.X_train):len(self.X_train)+len(self.X_val)]
        
        # test
        self.X_test = self.X[self.eoy_val:self.eoy_test]
        self.y_test = self.y[-len(self.X_test):]
        
        assert (len(self.X_train)+len(self.X_val)+len(self.X_test)) == len(self.data), \
            "sum of X train, val, test is not equal length of dataset"
        assert (len(self.y_train)+len(self.y_val)+len(self.y_test) == len(self.data)), \
        "sum of y train, val, test is not equal to length of dataset"
        
#         #standardize X_train
#         mean = torch.mean(self.X_train, axis=0)
#         std = torch.std(self.X_train, axis=0)
        
#         # Standardize X_train, X_val and X_test with mean/std from X_train
#         self.X_train = (self.X_train - mean) / std
#         self.X_val = (self.X_val - mean) / std
#         self.X_test = (self.X_test - mean) / std

        # Save variables
        # input dim
        self.input_dim = self.X_train.shape[1]
        # number of classes
        self.num_classes = len(np.unique(self.y_train))
#         class weights
        self.class_weights = len(self.y_train) / np.unique(self.y_train, return_counts=True)[1]
        
        print("*****************************************************************************************")
        print("Current dataset information:")
        print("---")
        print("class_weights:", self.class_weights)
        print("---")
        print(f"# of input data: {len(self.data)} with shape: {self.data.shape}")
        print(f"# of training samples: {len(self.y_train)} with X_train of shape: {self.X_train.shape}")
        print(f"# of validation samples: {len(self.y_val)} with X_val of shape: {self.X_val.shape}")
        print(f"# of test samples: {len(self.y_test)} with X_test of shape: {self.X_test.shape}")
        print("---")
        print(f"train start date: ", self.dates.iloc[0].strftime("%Y-%m-%d"), 
              ", train end date: ", self.dates.iloc[:self.eoy_train].iloc[-1].strftime("%Y-%m-%d"))
        print(f"val start date: ", self.dates.iloc[self.eoy_train:self.eoy_val].iloc[0].strftime("%Y-%m-%d"), 
              ", val end date: ", self.dates.iloc[self.eoy_train:self.eoy_val].iloc[-1].strftime("%Y-%m-%d"))
        print(f"test start date: ", self.dates.iloc[self.eoy_val:self.eoy_test].iloc[0].strftime("%Y-%m-%d"), 
              ", test end date: ", self.dates.iloc[self.eoy_val:self.eoy_test].iloc[-1].strftime("%Y-%m-%d"))
        print("*****************************************************************************************")
        
    def get_datasets(self):
        return self.X_train, self.X_val, self.X_test

In [8]:
def inner_lr_tune(config, data):
    # needed for reproducibility, will seed trainer (init of weights in NN?)
    pl.seed_everything(42, workers=True)

    data = data
    
    clf = make_pipeline(StandardScaler(), 
                        SGDClassifier(
                        max_iter=config["num_epochs"],
                        class_weight="balanced",
                        alpha=config["alpha"],
#                         epsilon=config["epsilon"],
                        tol=1e-3,
                        verbose=3,
                        n_jobs=-1,
                        ))
    
    clf.fit(data.X_train, data.y_train)
    
    y_pred = clf.predict(data.X_val)
    
    val_bal_acc = balanced_accuracy_score(data.y_val, y_pred)
    
    tune.report(val_bal_acc=val_bal_acc)

    return {"val_bal_acc": val_bal_acc}


def lr_tune():
    
    # Example parameters to tune from SGDClassifier
    config = {
                "alpha": tune.choice([1e-4, 1e-1, 1]),
                "num_epochs": tune.choice([100, 1000]),
#               "epsilon": [0.01, 0.1
             }

    scheduler = ASHAScheduler(
        max_t=1,
        grace_period=1,
        reduction_factor=2)

    reporter = CLIReporter(
        parameter_columns=["alpha"],
        metric_columns=["val_bal_acc"])
    
    data = Dataset()

    train_fn_with_parameters = tune.with_parameters(inner_lr_tune,
                                                    data=data,
#                                                     args=args,
#                                                     year_idx=year_idx,
#                                                     ckpt_path=ckpt_path,
                                                   )
    resources_per_trial = {"cpu": 8, "gpu": 0}

    # Set logging directory for tune.run
    log_dir = "./logs/tune/lr_loops"
    time = datetime.now().strftime("%Y%m%d%H%M%S")
#     # name = time+"_"+string_from_config(config) # config into path 
#     # CAREFUL: will give error if directory path is too large
#     train_year_end = 1996 + args.init_train_length + year_idx - 1
#     val_year_end = train_year_end + args.val_length
#     years = f"train{train_year_end}_val{val_year_end}"
    name = time

#     # save config space as .json
#     summary_path = Path.cwd()/log_dir/time
#     summary_path.mkdir(exist_ok=True, parents=True)
#     with open(summary_path/"config.json", 'w') as f:
#         json.dump(serialize_config(config), fp=f, indent=3)

#     # save args to json
#     args_dict = serialize_args(args.__dict__) #functions are not serializable
#     with open(summary_path/'args.json', 'w') as f:
#         json.dump(args_dict, f, indent=3)


    analysis = tune.run(train_fn_with_parameters,
        local_dir=log_dir,
        resources_per_trial=resources_per_trial,
        metric="val_bal_acc",
        mode="max",
        config=config,
        num_samples=10,
        scheduler=scheduler,
        progress_reporter=reporter,
        name=name,
        fail_fast=True, # stop all trials as soon as any trial errors
#         keep_checkpoints_num=1, # only keep best checkpoint
#         checkpoint_score_attr="min-val_loss",
        )

#     print("Best hyperparameters found were: ", analysis.best_config)
    
#     best_last_trial = analysis.get_best_trial("val_bal_acc", "max", "last") #change "last" to "all" for global min
# #     print("Best trial among last epoch config: {}".format(best_last_trial.config))
# #     print("Best trial >>last epoch<< validation loss: {}".format(
# #         best_last_trial.last_result["val_loss"]))
#     print("Best trial >>last epoch<< validation balanced accuracy: {}".format(
#         best_last_trial.last_result["val_bal_acc"]))
    

#     best_result_per_trial_df = analysis.dataframe(metric="val_loss", 
#                                             mode="min").sort_values("val_loss")
#     # save df to folder?
#     best_result = best_result_per_trial_df.iloc[0, :].to_dict() #take best values of best trial

#     # test prediction: best checkpoint out of all trials
#     best_trial = analysis.get_best_trial("val_loss", "min", scope="all")
#     if args.refit:
#         config = best_trial.config
#         ckpt_path = Path(analysis.get_best_checkpoint(best_trial).get_internal_representation()[1], "checkpoint")
    
#     if not args.no_predict:
#         best_path = Path(analysis.get_best_checkpoint(best_trial).get_internal_representation()[1], "checkpoint")
#         print(f"Loading model from path at {ckpt_path}")
#         model = FFN.load_from_checkpoint(best_path)
#         dm = MyDataModule_Loop(
#             path=args.path_data,
#             year_idx=year_idx,
#             dataset=args.dataset,
#             batch_size=1, #any number, doesnt matter for predict
#             init_train_length=args.init_train_length,
#             val_length=args.val_length,
#             label_fn=args.label_fn,
#             # config=model.hparams.config, # so that config is not hyperparam search again
#         )
#         trainer = pl.Trainer(
#             deterministic=True,
#             gpus=args.gpus_per_trial,
#         )
#         preds = trainer.predict(model=model, datamodule=dm)
#         preds_argmax = preds[0].argmax(dim=1).numpy() # assumes batchsize is whole testset
#         preds_argmax_df = pd.DataFrame(preds_argmax, columns=["pred"])
#         test_year_end = val_year_end + args.test_length
#         # prediction path
#         save_to_dir = Path(Path.cwd(),log_dir, name, f"prediction{test_year_end}.csv")
#         preds_argmax_df.to_csv(save_to_dir, index_label="id")
        
#     return best_result, summary_path, ckpt_path, config 
    #dictionary of best metrics and config, path
    return analysis


In [9]:
analysis = lr_tune()

*****************************************************************************************
Current dataset information:
---
class_weights: [1.56424512 2.77227938]
---
# of input data: 3823386 with shape: (3823386, 17)
# of training samples: 2780818 with X_train of shape: (2780818, 15)
# of validation samples: 647992 with X_val of shape: (647992, 15)
# of test samples: 394576 with X_test of shape: (394576, 15)
---
train start date:  1996-01-31 , train end date:  2018-12-31
val start date:  2019-01-31 , val end date:  2020-12-31
test start date:  2021-01-31 , test end date:  2021-11-30
*****************************************************************************************


2022-07-20 15:21:28,100	WARNING function_runner.py:603 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-07-20 15:21:28,232	WARNING tune.py:668 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
2022-07-20 15:21:28,403	ERROR syncer.py:147 -- Log sync requires rsync to be installed.


== Status ==
Current time: 2022-07-20 15:21:28 (running for 00:00:00.18)
Memory usage on this node: 8.9/15.9 GiB
Using FIFO scheduling algorithm.
Resources requested: 8.0/8 CPUs, 0/1 GPUs, 0.0/5.41 GiB heap, 0.0/2.71 GiB objects
Result logdir: C:\Users\Mathiass\Documents\Projects\master-thesis\notebooks\logs\tune\lr_loops\20220720152120
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------------+----------+----------------+---------+
| Trial name                | status   | loc            |   alpha |
|---------------------------+----------+----------------+---------|
| inner_lr_tune_dc9c1_00000 | RUNNING  | 127.0.0.1:5300 |  1      |
| inner_lr_tune_dc9c1_00001 | PENDING  |                |  0.1    |
| inner_lr_tune_dc9c1_00002 | PENDING  |                |  1      |
| inner_lr_tune_dc9c1_00003 | PENDING  |                |  0.1    |
| inner_lr_tune_dc9c1_00004 | PENDING  |                |  1      |
| inner_lr_tune_dc9c1_00005 | PENDING  |                |  1      |
| 

(inner_lr_tune pid=5300) Global seed set to 42


(inner_lr_tune pid=5300) -- Epoch 1
(inner_lr_tune pid=5300) Norm: 0.11, NNZs: 15, Bias: 0.012499, T: 2780818, Avg. loss: 0.992015
(inner_lr_tune pid=5300) Total training time: 0.57 seconds.
(inner_lr_tune pid=5300) -- Epoch 2
(inner_lr_tune pid=5300) Norm: 0.11, NNZs: 15, Bias: 0.012516, T: 5561636, Avg. loss: 0.992007
(inner_lr_tune pid=5300) Total training time: 1.16 seconds.
(inner_lr_tune pid=5300) -- Epoch 3
(inner_lr_tune pid=5300) Norm: 0.11, NNZs: 15, Bias: 0.012546, T: 8342454, Avg. loss: 0.992031
(inner_lr_tune pid=5300) Total training time: 1.73 seconds.
(inner_lr_tune pid=5300) -- Epoch 4
(inner_lr_tune pid=5300) Norm: 0.11, NNZs: 15, Bias: 0.012571, T: 11123272, Avg. loss: 0.992066
(inner_lr_tune pid=5300) Total training time: 2.29 seconds.
(inner_lr_tune pid=5300) -- Epoch 5
(inner_lr_tune pid=5300) Norm: 0.11, NNZs: 15, Bias: 0.012578, T: 13904090, Avg. loss: 0.992029
(inner_lr_tune pid=5300) Total training time: 2.85 seconds.
(inner_lr_tune pid=5300) -- Epoch 6
(inner_

...

(inner_lr_tune pid=5300) Global seed set to 42


(inner_lr_tune pid=5300) -- Epoch 1
(inner_lr_tune pid=5300) Norm: 0.54, NNZs: 15, Bias: 0.051532, T: 2780818, Avg. loss: 1.067967
(inner_lr_tune pid=5300) Total training time: 0.68 seconds.
(inner_lr_tune pid=5300) -- Epoch 2
(inner_lr_tune pid=5300) Norm: 0.63, NNZs: 15, Bias: 0.117019, T: 5561636, Avg. loss: 0.983015
(inner_lr_tune pid=5300) Total training time: 1.26 seconds.
(inner_lr_tune pid=5300) -- Epoch 3
(inner_lr_tune pid=5300) Norm: 0.56, NNZs: 15, Bias: 0.075266, T: 8342454, Avg. loss: 0.986419
(inner_lr_tune pid=5300) Total training time: 1.83 seconds.
(inner_lr_tune pid=5300) -- Epoch 4
(inner_lr_tune pid=5300) Norm: 0.61, NNZs: 15, Bias: 0.145016, T: 11123272, Avg. loss: 0.987935
(inner_lr_tune pid=5300) Total training time: 2.40 seconds.
(inner_lr_tune pid=5300) -- Epoch 5
(inner_lr_tune pid=5300) Norm: 0.62, NNZs: 15, Bias: 0.106584, T: 13904090, Avg. loss: 0.988992
(inner_lr_tune pid=5300) Total training time: 2.97 seconds.
(inner_lr_tune pid=5300) -- Epoch 6
(inner_

(inner_lr_tune pid=5300) Global seed set to 42


(inner_lr_tune pid=5300) -- Epoch 1
(inner_lr_tune pid=5300) Norm: 0.49, NNZs: 15, Bias: 0.105420, T: 2780818, Avg. loss: 0.982937
(inner_lr_tune pid=5300) Total training time: 0.71 seconds.
(inner_lr_tune pid=5300) -- Epoch 2
(inner_lr_tune pid=5300) Norm: 0.49, NNZs: 15, Bias: 0.103530, T: 5561636, Avg. loss: 0.984454
(inner_lr_tune pid=5300) Total training time: 1.48 seconds.
(inner_lr_tune pid=5300) -- Epoch 3
(inner_lr_tune pid=5300) Norm: 0.49, NNZs: 15, Bias: 0.103973, T: 8342454, Avg. loss: 0.984431
(inner_lr_tune pid=5300) Total training time: 2.19 seconds.
(inner_lr_tune pid=5300) -- Epoch 4
(inner_lr_tune pid=5300) Norm: 0.49, NNZs: 15, Bias: 0.103983, T: 11123272, Avg. loss: 0.984622
(inner_lr_tune pid=5300) Total training time: 2.87 seconds.
(inner_lr_tune pid=5300) -- Epoch 5
(inner_lr_tune pid=5300) Norm: 0.49, NNZs: 15, Bias: 0.104379, T: 13904090, Avg. loss: 0.984533
(inner_lr_tune pid=5300) Total training time: 3.56 seconds.
(inner_lr_tune pid=5300) -- Epoch 6
== Stat

2022-07-20 15:22:26,135	INFO tune.py:747 -- Total run time: 58.04 seconds (57.78 seconds for the tuning loop).
(pid=) 2022-07-20 15:22:26,183	INFO context.py:67 -- Exec'ing worker with command: "C:\Users\Mathiass\Anaconda3\envs\masterthesis\python.exe" C:\Users\Mathiass\Anaconda3\envs\masterthesis\lib\site-packages\ray\workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=61993 --object-store-name=tcp://127.0.0.1:64005 --raylet-name=tcp://127.0.0.1:64299 --redis-address=None --storage=None --temp-dir=C:\Users\Mathiass\AppData\Local\Temp\ray --metrics-agent-port=64240 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:63954 --redis-password=5241590000000000 --startup-token=8 --runtime-env-hash=137120697


In [10]:
# 60.82 seconds

In [11]:
analysis.results

{'dc9c1_00000': {'val_bal_acc': 0.518135881424077,
  'time_this_iter_s': 5.004813194274902,
  'done': True,
  'timesteps_total': None,
  'episodes_total': None,
  'training_iteration': 1,
  'trial_id': 'dc9c1_00000',
  'experiment_id': '4bbc8164df634919b5183aad5e458b1e',
  'date': '2022-07-20_15-21-38',
  'timestamp': 1658323298,
  'time_total_s': 5.004813194274902,
  'pid': 5300,
  'hostname': 'Mathias',
  'node_ip': '127.0.0.1',
  'config': {'alpha': 1, 'num_epochs': 100},
  'time_since_restore': 5.004813194274902,
  'timesteps_since_restore': 0,
  'iterations_since_restore': 1,
  'warmup_time': 0.005000591278076172,
  'experiment_tag': '0_alpha=1,num_epochs=100'},
 'dc9c1_00001': {'val_bal_acc': 0.5083311837995531,
  'time_this_iter_s': 5.13308048248291,
  'done': True,
  'timesteps_total': None,
  'episodes_total': None,
  'training_iteration': 1,
  'trial_id': 'dc9c1_00001',
  'experiment_id': '4bbc8164df634919b5183aad5e458b1e',
  'date': '2022-07-20_15-21-44',
  'timestamp': 1658

In [12]:
analysis.dataframe(metric="val_bal_acc", mode="max").sort_values("val_bal_acc")


,val_bal_acc,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,...,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time,config/alpha,config/num_epochs,logdir
1,0.508331,5.133080,False,NaN,NaN,1,dc9c1_00001,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-21-44,1658323304,...,5300,Mathias,127.0.0.1,5.133080,0,1,0.005001,0.1000,100,C:\Users\Mathiass\Documents\Projects\master-th...
3,0.508331,5.176682,False,NaN,NaN,1,dc9c1_00003,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-21-54,1658323314,...,5300,Mathias,127.0.0.1,5.176682,0,1,0.005001,0.1000,1000,C:\Users\Mathiass\Documents\Projects\master-th...
6,0.508331,4.878317,False,NaN,NaN,1,dc9c1_00006,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-22-09,1658323329,...,5300,Mathias,127.0.0.1,4.878317,0,1,0.005001,0.1000,100,C:\Users\Mathiass\Documents\Projects\master-th...
7,0.508331,4.895936,False,NaN,NaN,1,dc9c1_00007,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-22-14,1658323334,...,5300,Mathias,127.0.0.1,4.895936,0,1,0.005001,0.1000,100,C:\Users\Mathiass\Documents\Projects\master-th...
9,0.508331,5.726903,False,NaN,NaN,1,dc9c1_00009,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-22-25,1658323345,...,5300,Mathias,127.0.0.1,5.726903,0,1,0.005001,0.1000,100,C:\Users\Mathiass\Documents\Projects\master-th...
8,0.508394,5.591873,False,NaN,NaN,1,dc9c1_00008,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-22-20,1658323340,...,5300,Mathias,127.0.0.1,5.591873,0,1,0.005001,0.0001,100,C:\Users\Mathiass\Documents\Projects\master-th...
0,0.518136,5.004813,False,NaN,NaN,1,dc9c1_00000,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-21-38,1658323298,...,5300,Mathias,127.0.0.1,5.004813,0,1,0.005001,1.0000,100,C:\Users\Mathiass\Documents\Projects\master-th...
2,0.518136,5.199333,False,NaN,NaN,1,dc9c1_00002,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-21-49,1658323309,...,5300,Mathias,127.0.0.1,5.199333,0,1,0.005001,1.0000,100,C:\Users\Mathiass\Documents\Projects\master-th...
4,0.518136,4.954104,False,NaN,NaN,1,dc9c1_00004,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-21-59,1658323319,...,5300,Mathias,127.0.0.1,4.954104,0,1,0.005001,1.0000,1000,C:\Users\Mathiass\Documents\Projects\master-th...
5,0.518136,5.056656,False,NaN,NaN,1,dc9c1_00005,4bbc8164df634919b5183aad5e458b1e,2022-07-20_15-22-04,1658323324,...,5300,Mathias,127.0.0.1,5.056656,0,1,0.005001,1.0000,100,C:\Users\Mathiass\Documents\Projects\master-th...


In [13]:
analysis.get_best_trial("val_bal_acc", "max", "last")

inner_lr_tune_dc9c1_00000

In [14]:
# 2022-07-14 16:54:58,923	WARNING worker.py:1404 -- Warning: The actor ImplicitFunc is very large (74 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.